## Heston Parameter Calibration from options_data.csv
Load market call prices from `options_data.csv`, calibrate Heston parameters via gradient descent, and report the calibrated values.

In [1]:
import math
from typing import Dict, Tuple

import numpy as np
import pandas as pd
from numpy.linalg import norm

CSV_PATH = '../options_data.csv'
df = pd.read_csv(CSV_PATH)
required = {'S0', 'K', 'T', 'C_mkt'}
missing = required - set(df.columns)
if missing:
    raise ValueError(f'Missing columns: {missing}')
market_calls = df[['S0', 'K', 'T', 'C_mkt']].dropna().to_numpy()  # rows: [S0,K,T,price]
market_calls[:5]


array([[6.71929993e+02, 5.70000000e+02, 4.06529346e-03, 9.37800000e+01],
       [6.71929993e+02, 5.75000000e+02, 4.06529346e-03, 1.02280000e+02],
       [6.71929993e+02, 5.80000000e+02, 4.06529346e-03, 8.56200000e+01],
       [6.71929993e+02, 5.90000000e+02, 4.06529346e-03, 7.56500000e+01],
       [6.71929993e+02, 6.00000000e+02, 4.06529346e-03, 7.13200000e+01]])

In [2]:
PHI_MAX = 200.0
PHI_STEPS = 2001

def heston_probability(S0, K, T, r, params, Pnum):
    phi = np.linspace(1e-5, PHI_MAX, PHI_STEPS)
    kappa, theta, sigma, rho, v0 = params
    u = 0.5 if Pnum == 1 else -0.5
    b = kappa - rho * sigma if Pnum == 1 else kappa
    a = kappa * theta
    x = math.log(S0)
    d = np.sqrt((rho * sigma * 1j * phi - b) ** 2 - sigma ** 2 * (2 * u * 1j * phi - phi ** 2))
    g = (b - rho * sigma * 1j * phi + d) / (b - rho * sigma * 1j * phi - d)
    exp_dt = np.exp(-d * T)
    log_term = np.log((1.0 - g * exp_dt) / (1.0 - g))
    C = r * 1j * phi * T + (a / (sigma ** 2)) * ((b - rho * sigma * 1j * phi + d) * T - 2.0 * log_term)
    D = ((b - rho * sigma * 1j * phi + d) / (sigma ** 2)) * ((1.0 - exp_dt) / (1.0 - g * exp_dt))
    integrand = np.real(np.exp(C + D * v0 + 1j * phi * (x - math.log(K))) / (1j * phi))
    h = phi[1] - phi[0]
    integral = h / 3.0 * (integrand[0] + integrand[-1] + 4.0 * np.sum(integrand[1:-1:2]) + 2.0 * np.sum(integrand[2:-2:2]))
    return 0.5 + (1.0 / math.pi) * integral

def heston_call_price(S0, K, T, r, params):
    P1 = heston_probability(S0, K, T, r, params, 1)
    P2 = heston_probability(S0, K, T, r, params, 2)
    return S0 * P1 - K * math.exp(-r * T) * P2

def call_price_vector(params, market_data, r):
    return np.array([heston_call_price(S0, K, T, r, params) for S0, K, T, _ in market_data])


In [3]:
def loss(params, market_data, r):
    prices = call_price_vector(params, market_data, r)
    diffs = prices - market_data[:, 3]
    return 0.5 * np.mean(diffs ** 2)

def numerical_grad(params, market_data, r, eps=1e-4):
    grad = np.zeros_like(params)
    base = loss(params, market_data, r)
    for i in range(len(params)):
        shift = np.zeros_like(params)
        shift[i] = eps
        params_up = params + shift
        grad[i] = (loss(params_up, market_data, r) - base) / eps
    return grad


In [4]:
# Calibration loop
risk_free_rate = 0.02
params = np.array([1.5, 0.04, 0.5, -0.5, 0.04], dtype=float)  # kappa, theta, sigma, rho, v0
learning_rate = 1e-3
max_iters = 30
history = []
for it in range(max_iters):
    grad = numerical_grad(params, market_calls, risk_free_rate)
    params -= learning_rate * grad
    history.append((it, loss(params, market_calls, risk_free_rate)))
history[:5]


/tmp/ipykernel_510/2940631294.py:12: RuntimeWarning: divide by zero encountered in divide
  g = (b - rho * sigma * 1j * phi + d) / (b - rho * sigma * 1j * phi - d)
/tmp/ipykernel_510/2940631294.py:14: RuntimeWarning: invalid value encountered in multiply
  log_term = np.log((1.0 - g * exp_dt) / (1.0 - g))
/tmp/ipykernel_510/2940631294.py:14: RuntimeWarning: invalid value encountered in divide
  log_term = np.log((1.0 - g * exp_dt) / (1.0 - g))
/tmp/ipykernel_510/2940631294.py:16: RuntimeWarning: invalid value encountered in multiply
  D = ((b - rho * sigma * 1j * phi + d) / (sigma ** 2)) * ((1.0 - exp_dt) / (1.0 - g * exp_dt))
/tmp/ipykernel_510/2940631294.py:16: RuntimeWarning: invalid value encountered in divide
  D = ((b - rho * sigma * 1j * phi + d) / (sigma ** 2)) * ((1.0 - exp_dt) / (1.0 - g * exp_dt))
/tmp/ipykernel_510/2940631294.py:17: RuntimeWarning: overflow encountered in exp
  integrand = np.real(np.exp(C + D * v0 + 1j * phi * (x - math.log(K))) / (1j * phi))
/tmp/ipykerne

[(0, np.float64(nan)),
 (1, np.float64(nan)),
 (2, np.float64(nan)),
 (3, np.float64(nan)),
 (4, np.float64(nan))]

In [5]:
result = pd.DataFrame({
    'Iteration': [h[0] for h in history],
    'Loss': [h[1] for h in history]
})
result.tail()


,Iteration,Loss
25,25,NaN
26,26,NaN
27,27,NaN
28,28,NaN
29,29,NaN


In [6]:
calibrated = pd.Series(params, index=['kappa', 'theta', 'sigma', 'rho', 'v0'])
calibrated


kappa   NaN
theta   NaN
sigma   NaN
rho     NaN
v0      NaN
dtype: float64